In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1rc3.dev49
cdv, version 1.1.2
Python 3.10.8


In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)

from chia.consensus.default_constants import DEFAULT_CONSTANTS
from chia.types.blockchain_format.coin import Coin
from chia.types.blockchain_format.program import Program
from chia.types.coin_spend import CoinSpend
from chia.types.condition_opcodes import ConditionOpcode
from chia.types.spend_bundle import SpendBundle
from chia.util.hash import std_hash
from chia.wallet.puzzles import p2_delegated_puzzle_or_hidden_puzzle

from clvm.casts import int_to_bytes
from clvm_tools.clvmc import compile_clvm_text
from clvm_tools.binutils import disassemble

# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle, print_json, print_push_tx_result)


## Coin Announcement

#### `CREATE_COIN_ANNOUNCEMENT`	
`(60 message)`	Creates a block announcement specific to this coin.

#### `ASSERT_COIN_ANNOUNCEMENT`	
`(61 announcement_id)`	Requires a coin specific block announcement by its id.

`announcement_id == sha256(coinID + message)`

In [3]:
announcer_puzzle = Program(
    compile_clvm_text(
'''
(mod (announcement)
    (list
        (list 60 announcement)
    )
)
''', search_paths=[]
    )
)

asserter_puzzle = Program(
    compile_clvm_text(
'''
(mod (announcer_coin_id announcement)
    (list
        (list 61 (sha256 announcer_coin_id announcement))
    )
)
''', search_paths=[]
    )
)

print_program(announcer_puzzle)
print_program(asserter_puzzle)

(c (c (q . 60) (c 2 ())) ())
(c (c (q . 61) (c (sha256 2 5) ())) ())


In [7]:
parent_coin_info = bytes.fromhex("f85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1")

announcer_coin = Coin(
    parent_coin_info,
    announcer_puzzle.get_tree_hash(),
    1
)

message = bytes("hello", "utf-8")
announcement = std_hash(message)
solution = Program.to([announcement])

result = announcer_puzzle.run(solution)
print_program(result)

announcer_coin_spend = CoinSpend(
    announcer_coin,
    announcer_puzzle,
    solution
)

print_json(announcer_coin_spend.to_json_dict())

((60 0x2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824))
{
    "coin": {
        "amount": 1,
        "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
        "puzzle_hash": "0xfe2f967d127f57fd68395ad068c99d3e138d129e93dcb0ee69ddcb9fc6982773"
    },
    "puzzle_reveal": "0xff04ffff04ffff013cffff04ff02ff808080ff8080",
    "solution": "0xffa02cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b982480"
}


In [8]:
asserter_coin = Coin(
    parent_coin_info,
    asserter_puzzle.get_tree_hash(),
    1
)

message = bytes("hello", "utf-8")
announcement = std_hash(message)
solution = Program.to([announcer_coin.name(), announcement])

result = asserter_puzzle.run(solution)
print_program(result)

asserter_coin_spend = CoinSpend(
    asserter_coin,
    asserter_puzzle,
    solution
)

print_json(asserter_coin_spend.to_json_dict())

((61 0xd2eff3682669e021371cc956264cde255082136e4e3dddd1dbce1e0d9d5467c1))
{
    "coin": {
        "amount": 1,
        "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
        "puzzle_hash": "0x42cc0686c5a4491c0e5aa8d687556344f2ac0d7aade8b6932f2c6907f6d529ff"
    },
    "puzzle_reveal": "0xff04ffff04ffff013dffff04ffff0bff02ff0580ff808080ff8080",
    "solution": "0xffa00f9229c339e54bc8da0a244c34b370164d14cb63fede6d91d30351b96ad50ad2ffa02cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b982480"
}


In [9]:
spend_bundle = SpendBundle(
    [
        announcer_coin_spend,
        asserter_coin_spend
    ],
    G2Element()
)

print_json(spend_bundle.to_json_dict())

{
    "aggregated_signature": "0xc00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
    "coin_solutions": [
        {
            "coin": {
                "amount": 1,
                "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
                "puzzle_hash": "0xfe2f967d127f57fd68395ad068c99d3e138d129e93dcb0ee69ddcb9fc6982773"
            },
            "puzzle_reveal": "0xff04ffff04ffff013cffff04ff02ff808080ff8080",
            "solution": "0xffa02cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b982480"
        },
        {
            "coin": {
                "amount": 1,
                "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
                "puzzle_hash": "0x42cc0686c5a4491c0e5aa8d687556344f2ac0d7aade8b6932f2c6907f6d529ff"
            

### Debug Spend Bundle

```sh
❯ cdv inspect spendbundles $spendbundle -db
...
brun -y main.sym '(c (c (q . 60) (c 2 ())) ())' '(0x2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824)'

((CREATE_COIN_ANNOUNCEMENT 0x2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824))

grouped conditions:

  (CREATE_COIN_ANNOUNCEMENT 0x2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824)


-------
consuming coin (0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1 0x42cc0686c5a4491c0e5aa8d687556344f2ac0d7aade8b6932f2c6907f6d529ff 1)
  with id ee9a9818a8769bd85f55353f4956fd4676c01d39a95ff5f51a31f20e446b7d0d


brun -y main.sym '(c (c (q . 61) (c (sha256 2 5) ())) ())' '(0x0f9229c339e54bc8da0a244c34b370164d14cb63fede6d91d30351b96ad50ad2 0x2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824)'

((ASSERT_COIN_ANNOUNCEMENT 0xd2eff3682669e021371cc956264cde255082136e4e3dddd1dbce1e0d9d5467c1))

grouped conditions:

  (ASSERT_COIN_ANNOUNCEMENT 0xd2eff3682669e021371cc956264cde255082136e4e3dddd1dbce1e0d9d5467c1)


-------

spent coins
  (0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1 0xfe2f967d127f57fd68395ad068c99d3e138d129e93dcb0ee69ddcb9fc6982773 1)
      => spent coin id 0f9229c339e54bc8da0a244c34b370164d14cb63fede6d91d30351b96ad50ad2
  (0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1 0x42cc0686c5a4491c0e5aa8d687556344f2ac0d7aade8b6932f2c6907f6d529ff 1)
      => spent coin id ee9a9818a8769bd85f55353f4956fd4676c01d39a95ff5f51a31f20e446b7d0d

created coins
created coin announcements
  ['0x0f9229c339e54bc8da0a244c34b370164d14cb63fede6d91d30351b96ad50ad2', '0x2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824'] =>
      d2eff3682669e021371cc956264cde255082136e4e3dddd1dbce1e0d9d5467c1


zero_coin_set = []

created  coin announcements = ['d2eff3682669e021371cc956264cde255082136e4e3dddd1dbce1e0d9d5467c1']

asserted coin announcements = ['d2eff3682669e021371cc956264cde255082136e4e3dddd1dbce1e0d9d5467c1']

symdiff of coin announcements = []


================================================================================

aggregated signature check pass: True
pks: []
msgs: []
  msg_data: []
  coin_ids: []
  add_data: []
signature: c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
None
```